<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.07-Merge-and-Join.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# データセットの結合: マージと結合

Pandas が提供する重要な機能の 1 つは、その高性能なメモリ内結合およびマージ操作です。
データベースを扱ったことがある場合は、このタイプのデータ相互作用に精通しているはずです。
このための主なインターフェースは ``pd.merge`` 関数であり、これが実際にどのように機能するかの例をいくつか見ていきます。

便宜上、前のセクションの ``display()`` 機能を再定義することから始めます:

In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 関係代数

``pd.merge()`` で実装された動作は、リレーショナル データを操作するための正式なルール セットである *リレーショナル代数* として知られているもののサブセットであり、ほとんどのデータベースで利用可能な操作の概念的基盤を形成します。
リレーショナル代数アプローチの強みは、いくつかの基本的な操作を提案することです。これらの操作は、任意のデータセットに対するより複雑な操作の構成要素になります。
データベースまたは他のプログラムに効率的に実装された基本操作のこの用語集を使用すると、かなり複雑な複合操作を幅広く実行できます。

Pandas は、 pd.merge() 関数と、関連する Series および Dataframe の join() メソッドで、これらの基本的な構成要素のいくつかを実装しています。
これから説明するように、これらを使用すると、さまざまなソースからのデータを効率的にリンクできます。

## 結合のカテゴリ

``pd.merge()`` 関数は、*1 対 1*、*多対 1*、*多対多*の結合など、さまざまなタイプの結合を実装します。
3 種類の結合はすべて、 pd.merge() インターフェースへの同一の呼び出しを介してアクセスされます。実行される結合のタイプは、入力データの形式によって異なります。
ここでは、3 種類のマージの簡単な例を示し、詳細なオプションについては後で説明します。

### 1 対 1 の結合

おそらく、最も単純なタイプのマージ式は 1 対 1 の結合です。これは、[Combining Datasets: Concat & Append](03.06-Concat-And-Append.ipynb) で見られる列単位の連結に多くの点で非常に似ています。
具体的な例として、会社の複数の従業員に関する情報を含む次の 2 つの ``DataFrame`` を考えてみましょう:

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


この情報を単一の ``DataFrame`` に結合するには、``pd.merge()`` 関数を使用できます。

In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


``pd.merge()`` 関数は、各 ``DataFrame`` に「employee」列があることを認識し、この列をキーとして自動的に結合します。
マージの結果は、2 つの入力からの情報を組み合わせた新しい ``DataFrame`` です。
各列のエントリの順序は必ずしも維持されないことに注意してください。この場合、「従業員」列の順序は、「df1」と「df2」、および「pd.merge()」との間で異なります。 ` 関数はこれを正しく説明します。
さらに、インデックスによるマージの特殊なケースを除いて、マージは一般にインデックスを破棄することに注意してください (一時的に議論される ``left_index`` および ``right_index`` キーワードを参照してください)。

### 多対一結合

多対 1 結合は、2 つのキー列の 1 つに重複するエントリが含まれる結合です。
多対一の場合、結果の ``DataFrame`` は重複したエントリを適切に保持します。
次の多対 1 結合の例を考えてみましょう。

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

結果として得られる ``DataFrame`` には、「スーパーバイザ」情報を含む追加の列があり、この情報は、入力の必要に応じて 1 つ以上の場所で繰り返されます。

### 多対多結合

多対多の結合は概念的に少し混乱しますが、それでも明確に定義されています。
左と右の両方の配列のキー列に重複が含まれている場合、結果は多対多のマージになります。
これはおそらく、具体的な例で最も明確になります。
特定のグループに関連付けられた 1 つまたは複数のスキルを示す ``DataFrame`` がある次の例を検討してください。
多対多の結合を実行することで、個人に関連付けられたスキルを回復できます。

In [5]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


これら 3 種類の結合を他の Pandas ツールで使用して、さまざまな機能を実装できます。
しかし実際には、データセットがここで扱っているものほどクリーンであることはめったにありません。
次のセクションでは、結合操作の動作を調整できる pd.merge() によって提供されるオプションのいくつかを検討します。

## マージキーの仕様

``pd.merge()`` のデフォルトの動作はすでに見ました: 2 つの入力の間で 1 つ以上の一致する列名を探し、これをキーとして使用します。
しかし、多くの場合、列名はうまく一致しません。 pd.merge() は、これを処理するためのさまざまなオプションを提供します。

### ``on`` キーワード

最も簡単な方法として、列名または列名のリストを受け取る ``on`` キーワードを使用して、キー列の名前を明示的に指定できます。

In [6]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

このオプションは、左右の ``DataFrame`` の両方に指定された列名がある場合にのみ機能します。

### ``left_on`` と ``right_on`` キーワード

列名が異なる 2 つのデータセットをマージしたい場合があります。たとえば、従業員名が「employee」ではなく「name」とラベル付けされているデータセットがあるとします。
この場合、 ``left_on`` および ``right_on`` キーワードを使用して、2 つの列名を指定できます。

In [7]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

結果には、必要に応じて削除できる冗長な列があります。たとえば、DataFrame の drop() メソッドを使用します。

In [8]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### ``left_index`` と ``right_index`` キーワード

場合によっては、列でマージするのではなく、インデックスでマージしたいことがあります。
たとえば、データは次のようになります。

In [9]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


``pd.merge()`` で ``left_index`` および/または ``right_index`` フラグを指定することにより、インデックスをマージのキーとして使用できます。

In [10]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


便宜上、DataFrame は join() メソッドを実装しています。これは、デフォルトでインデックスの結合を行うマージを実行します。

In [11]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


インデックスと列を混在させたい場合は、 left_index と right_on 、または left_on と right_index を組み合わせて、目的の動作を得ることができます。

In [12]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

これらのオプションはすべて、複数のインデックスや複数の列でも機能します。この動作のインターフェイスは非常に直感的です。
詳細については、Pandas ドキュメントの ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html) を参照してください。

## ジョインの集合演算の指定

前述のすべての例で、結合を実行する際の重要な考慮事項の 1 つ、結合で使用される集合演算の種類について説明しました。
これは、値が 1 つのキー列に表示され、他のキー列には表示されない場合に発生します。次の例を検討してください。

In [13]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

ここでは、共通の「名前」エントリが 1 つしかない 2 つのデータセットをマージしました: Mary.
デフォルトでは、結果には 2 つの入力セットの *交差* が含まれます。これは *内部結合* と呼ばれるものです。
``how`` キーワードを使用してこれを明示的に指定できます。デフォルトは ``"inner"`` です:

In [14]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


``how`` キーワードのその他のオプションは、``'outer'``、``'left'``、および ``'right'`` です。
*outer join* は、入力列の結合に対する結合を返し、欠損値をすべて NA で埋めます。

In [15]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

*left join* と *right join* は、それぞれ左エントリと右エントリの結合を返します。
例えば：

In [16]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

出力行は、左側の入力のエントリに対応するようになりました。使用する
``how='right'`` も同様の方法で機能します。

これらのオプションはすべて、前述の結合タイプのいずれにも直接適用できます。

## 列名の重複: ``suffixes`` キーワード

最後に、2 つの入力 ``DataFrame`` の列名が競合する場合があります。
次の例を検討してください。

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

出力には 2 つの競合する列名が含まれるため、マージ関数は自動的に接尾辞 ``_x`` または ``_y`` を追加して、出力列を一意にします。
これらのデフォルトが不適切な場合、 ``suffixes`` キーワードを使用してカスタム サフィックスを指定できます。

In [18]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

これらの接尾辞は、可能な結合パターンのいずれかで機能し、重複する列が複数ある場合にも機能します。

これらのパターンの詳細については、[Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) を参照してください。ここでは、リレーショナル代数をもう少し深く掘り下げています。
これらのトピックの詳細については、[Pandas "Merge, Join and Concatenate" documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html) も参照してください。

## 例: 米国の州のデータ

異なるソースからのデータを結合する場合、マージおよび結合操作が最も頻繁に発生します。
ここでは、米国の州とその人口に関するデータの例を考えます。
データ ファイルは http://github.com/jakevdp/data-USstates/ にあります。

In [19]:
# Following are shell commands to download the data
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Pandas ``read_csv()`` 関数を使用して、3 つのデータセットを見てみましょう。

In [20]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

この情報を基に、比較的単純な結果を計算したいとします。2010 年の人口密度で米国の州と準州をランク付けします。
この結果を見つけるためのデータがここにあることは明らかですが、結果を見つけるにはデータセットを結合する必要があります。

母集団 ``DataFrame`` 内の完全な状態名を取得する多対 1 のマージから始めます。
``pop`` の ``state/region`` 列と ``abbrevs`` の ``abbreviation`` 列に基づいてマージしたいと考えています。
``how='outer'`` を使用して、ラベルの不一致が原因でデータが破棄されないようにします。

In [21]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


ここで不一致があるかどうかを再確認しましょう。これは、null を含む行を探すことで実行できます。

In [22]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

一部の ``population`` 情報が null です。これらがどれであるかを考えてみましょう！

In [23]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


NULL 人口値はすべて 2000 年より前のプエルトリコのもののようです。これは、このデータが元のソースから入手できないことが原因である可能性があります。

さらに重要なことに、新しい ``state`` エントリのいくつかも null であることがわかります。これは、``abbrevs`` キーに対応するエントリがなかったことを意味します!
この一致がない地域を特定してみましょう。

In [24]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

この問題はすぐに推測できます。人口データには、プエルトリコ (PR) と米国全体 (USA) のエントリが含まれていますが、これらのエントリは州の略語キーには表示されません。
適切なエントリを入力することで、これらをすばやく修正できます。

In [25]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

``state`` 列に null はもうありません: これで準備完了です!

これで、同様の手順を使用して、結果をエリア データとマージできます。
結果を調べると、両方の ``state`` 列で結合したいと思うでしょう:

In [26]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


もう一度、null をチェックして、不一致がないかどうかを確認しましょう。

In [27]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

``area`` 列に null があります。ここで無視されたリージョンを確認できます。

In [28]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

``areas`` ``DataFrame`` には、米国全体の領域が含まれていないことがわかります。
適切な値を挿入することもできますが (たとえば、すべての州の合計を使用して)、米国全体の人口密度は現在の議論とは関係がないため、この場合は null 値を削除します。

In [29]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


これで、必要なデータがすべて揃いました。関心のある質問に答えるために、まず 2000 年に対応するデータの部分と総人口を選択しましょう。
``query()`` 関数を使用してこれを素早く行います (これには ``numexpr`` パッケージがインストールされている必要があります; [High-Performance Pandas: ``eval() と ``query()`` を参照してください](03.12-Performance-Eval -and-Query.ipynb)):

In [30]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


それでは、人口密度を計算して順番に表示してみましょう。
状態のデータを再インデックスすることから始めて、結果を計算します。

In [31]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [32]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

その結果は、2010 年の人口密度 (1 平方マイルあたりの居住者数) の順に、米国の州とワシントン DC、およびプエルトリコをランキングしたものです。
このデータセットで群を抜いて密度の高い地域はワシントン DC (つまり、コロンビア特別区) であることがわかります。州の中で最も密度が高いのはニュージャージー州です。

リストの最後を確認することもできます。

In [33]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

群を抜いて人口密度が最も低い州はアラスカ州であり、平均して 1 平方マイルあたりの居住者数は 1 人をわずかに上回っています。

この種の厄介なデータ マージは、実際のデータ ソースを使用して質問に答えようとするときによくあるタスクです。
この例で、データから洞察を得るために、これまでに説明したツールを組み合わせる方法についてのアイデアが得られたことを願っています!

<!--ナビゲーション-->
< [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.07-Merge-and-Join.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
